In [ ]:
pip install numpy pandas matplotlib seaborn scikit-learn yfinance keras tensorflow


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Fetching stock data for a specific company (e.g., Apple)
ticker = 'AAPL'
data = yf.download(ticker, start="2010-01-01", end="2023-01-01")

# Display first few rows
data.head()


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.447413,6.462175,6.398307,6.429939,493729600
2010-01-05,6.458559,6.495012,6.424516,6.465187,601904800
2010-01-06,6.355828,6.484168,6.349200,6.458560,552160000
2010-01-07,6.344077,6.386857,6.297983,6.379325,477131200
2010-01-08,6.386254,6.386857,6.298285,6.335642,447610800


In [2]:
import requests

def fetch_news(api_key, query='latest news'):
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        return []

# Usage
api_key = 'YOUR_API_KEY'
articles = fetch_news(api_key)


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Clean the articles
df = pd.DataFrame(articles)
df['cleaned_text'] = df['content'].apply(clean_text)


KeyError: 'content'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_text']).toarray()


Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - loss: 0.0168
Epoch 2/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - loss: 0.0018
Epoch 3/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - loss: 0.0013
Epoch 4/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - loss: 0.0015
Epoch 5/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - loss: 0.0013
Epoch 6/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - loss: 0.0011
Epoch 7/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - loss: 0.0010
Epoch 8/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - loss: 0.0011
Epoch 9/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0012
Epoch 10/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.0010
Epoch 11/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - loss: 9.6704e-04
Epoch 12/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - loss: 0.0011
Epoch 13/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - loss: 0.0010
Epoch 14/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 9.2052e-04
Epoch 15/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 10s

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have a dataframe `df` with 'label' column (1 for real, 0 for fake)
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, random_state=42)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
Predicted Price: [[136.20479]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


In [ ]:
def predict_article(article_text):
    cleaned_text = clean_text(article_text)
    vectorized_text = tfidf.transform([cleaned_text]).toarray()
    prediction = model.predict(vectorized_text)
    return 'Fake News' if prediction[0] == 0 else 'Real News'

# Example usage
new_article = "Your article content here."
print(predict_article(new_article))
